In [1]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.model_selection import train_test_split
from lazypredict.Supervised import LazyRegressor
from sklearn.utils import all_estimators
from sklearn.base import RegressorMixin

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

In [4]:
cols_cible_type = ["TurnoutTimeSeconds", "TravelTimeSeconds", "PumpSecondsOnSite"]
cols_Data = [
    ["CalYear", "HourOfCall", "Postcode_district", "Month", "DayOfWeek"],
    ["CalYear", "HourOfCall", "Postcode_district", "Month", "DayOfWeek"],
    ["CalYear", "PropertyType", "StopCode"],
]
cols_cible = [
    ["TurnoutTimeSeconds_min", "TurnoutTimeSeconds_mean", "TurnoutTimeSeconds_max"],
    ["TravelTimeSeconds_min", "TravelTimeSeconds_mean", "TravelTimeSeconds_max"],
    ["PumpSecondsOnSite_min", "PumpSecondsOnSite_mean", "PumpSecondsOnSite_max"]
    # NumPumpsAttending??
]
cols_cible_filter = [
    "TurnoutTimeSeconds_mean",
    "TravelTimeSeconds_mean",
    "PumpSecondsOnSite_mean",
]

def load_df(col_cible_type):
    df = pd.read_csv(f"../data/_df_ready_{col_cible_type}.csv", sep=";", low_memory=False)
    # Contrôle
    # display(df.head(3))
    # display(df.info())
    return df

In [5]:
# Créez le cercle de corrélation
# def draw_correlation_circle(df_charges_factorielles, pca, arrow_length=0.1, label_rotation=0):
#     fig, ax = plt.subplots(figsize=(8, 8))
#     for i, var in enumerate(df_charges_factorielles.columns):
#         x = df_charges_factorielles.loc[0, var]
#         y = df_charges_factorielles.loc[1, var]
#         ax.arrow(0, 0, x, y, head_width=arrow_length, head_length=arrow_length, fc='gray', ec='gray')
#         ax.text(x*1.15, y*1.15, var, ha='center', va='center', rotation=label_rotation, fontsize=9)
#     circle = plt.Circle((0, 0), 1, facecolor='none', edgecolor='black')
#     ax.add_artist(circle)
#     ax.set_xlim(-1.1, 1.1)
#     ax.set_ylim(-1.1, 1.1)
#     ax.set_aspect('equal', adjustable='box')
#     ax.set_xlabel('Axe 1 (PC1)')
#     ax.set_ylabel('Axe 2 (PC2)')
#     ax.set_title('Cercle des Corrélations')
#     plt.grid()
#     plt.show()

ignore_regressors = [
    "ExtraTreesRegressor",
    "NuSVR",
    "QuantileRegressor",
    "SVR",
    "CCA",
    "GammaRegressor",
    "GaussianProcessRegressor",
    "IsotonicRegression",
    "MultiOutputRegressor",
    "MultiTaskElasticNet",
    "MultiTaskElasticNetCV",
    "MultiTaskLasso",
    "MultiTaskLassoCV",
    "PLSCanonical",
    "RadiusNeighborsRegressor",
    "RegressorChain",
    "StackingRegressor",
    "KernelRidge",
    "ARDRegression",
    "TweedieRegressor",
]
REGRESSORS = [
    est
    for est in all_estimators()
    if (issubclass(est[1], RegressorMixin) and (not (est[0] in ignore_regressors)))
]

all_results = pd.DataFrame()

for index, name in enumerate(cols_cible_type):
    df = load_df(name)
    # X = df.drop(["PumpSecondsOnSite_min", "PumpSecondsOnSite_mean", "PumpSecondsOnSite_max", "TurnoutTimeSeconds_min", "TurnoutTimeSeconds_mean",
    #               "TurnoutTimeSeconds_max", "TravelTimeSeconds_min", "TravelTimeSeconds_mean", "TravelTimeSeconds_max", "NumPumpsAttending"], axis=1)
    # display(X.head(2))
    # print(X.shape)

    # pca = PCA()
    # data_2D = pca.fit_transform(X)
    # display(data_2D)
    # print("La part de variance expliquée est", round(pca.explained_variance_ratio_.sum(), 5))

    # plt.figure()
    # plt.xlim(0,40)
    # plt.xlabel('Nombre de composantes')
    # plt.ylabel('Part de variance expliquée')
    # plt.axhline(y = 0.95, color ='r', linestyle = '--')
    # plt.plot(pca.explained_variance_ratio_.cumsum());
    # plt.show()

    # variance_expliquee = pca.explained_variance_ratio_
    # charges_factorielles = pca.components_
    # # Créez un DataFrame pour afficher les corrélations de chaque variable avec chaque axe
    # df_charges_factorielles = pd.DataFrame(
    #     charges_factorielles,
    #     columns=X.columns,
    #     #index=["Axe 1", "Axe 2", "Axe 3", "Axe 4", "Axe 5", "Axe 6"],
    # )

    # # Affichez le DataFrame des charges factorielles
    # display(df_charges_factorielles.head(2))

    # charges_factorielles = pca.components_
    # # Appelez la fonction pour tracer le cercle de corrélation
    # draw_correlation_circle(df_charges_factorielles, pca)

    # Auto ML
    df_limited = df[df.CalYear > 6]
    print(df_limited.shape)
    # Crée X sans les target
    cols_to_remove = [item for sublist in cols_cible for item in sublist] + [
        "NumPumpsAttending"
    ]
    print("cols_to_remove", cols_to_remove)
    X = df_limited.drop(cols_to_remove, axis=1)
    # Ne conserve que certaines colonnes explicatives
    # Crée une liste des colonnes à conserver
    cols_to_keep = [
        col
        for col in X.columns
        if any(substring in col for substring in cols_Data[index])
    ]
    print("cols_to_keep", cols_to_keep)
    # Conserver uniquement ces colonnes
    X = X[cols_to_keep]

    display(X.head(2))
    print(X.shape)

    for col_cible in cols_cible[index]:
        if not(col_cible in cols_cible_filter):
            continue
        print("cible", col_cible)
        y = df_limited[col_cible]
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
        myREGRESSORS = REGRESSORS
        print(myREGRESSORS)
        # myREGRESSORS = myREGRESSORS[4:6]
        reg = LazyRegressor(
            verbose=2,
            ignore_warnings=False,
            custom_metric=None,
            regressors=myREGRESSORS,
        )
        models, predictions = reg.fit(X_train, X_test, y_train, y_test)
        # Ajoute le nom de la colonne
        models["Target"] = col_cible
        # Concaténe dans les résultats
        all_results = pd.concat([all_results, models], axis=0)
        display(models)
        models.to_csv(f"../data/_autoML_regressor_{col_cible}.csv", sep=";", index=True)
        # break
    # break

all_results.to_csv(f"../data/_autoML_regressor.csv", sep=";", index=True)

# tester automl avant et après PCA et aussi gridsearch pour comparer
# save du PCA

(1016665, 42)
cols_to_remove ['TurnoutTimeSeconds_min', 'TurnoutTimeSeconds_mean', 'TurnoutTimeSeconds_max', 'TravelTimeSeconds_min', 'TravelTimeSeconds_mean', 'TravelTimeSeconds_max', 'PumpSecondsOnSite_min', 'PumpSecondsOnSite_mean', 'PumpSecondsOnSite_max', 'NumPumpsAttending']
cols_to_keep ['CalYear', 'HourOfCall_0', 'HourOfCall_1', 'HourOfCall_2', 'HourOfCall_3', 'HourOfCall_4', 'Postcode_district_0', 'Postcode_district_1', 'Postcode_district_2', 'Postcode_district_3', 'Postcode_district_4', 'StopCode_0', 'StopCode_1', 'StopCode_2', 'StopCode_3', 'StopCode_4', 'Month_0', 'Month_1', 'Month_2', 'Month_3', 'DayOfWeek_0', 'DayOfWeek_1', 'DayOfWeek_2']


,CalYear,HourOfCall_0,HourOfCall_1,HourOfCall_2,HourOfCall_3,HourOfCall_4,Postcode_district_0,Postcode_district_1,Postcode_district_2,Postcode_district_3,Postcode_district_4,StopCode_0,StopCode_1,StopCode_2,StopCode_3,StopCode_4,Month_0,Month_1,Month_2,Month_3,DayOfWeek_0,DayOfWeek_1,DayOfWeek_2
574862,7,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,1
574863,7,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,1


(1016665, 23)
cible TurnoutTimeSeconds_mean
[('AdaBoostRegressor', <class 'sklearn.ensemble._weight_boosting.AdaBoostRegressor'>), ('BaggingRegressor', <class 'sklearn.ensemble._bagging.BaggingRegressor'>), ('BayesianRidge', <class 'sklearn.linear_model._bayes.BayesianRidge'>), ('DecisionTreeRegressor', <class 'sklearn.tree._classes.DecisionTreeRegressor'>), ('DummyRegressor', <class 'sklearn.dummy.DummyRegressor'>), ('ElasticNet', <class 'sklearn.linear_model._coordinate_descent.ElasticNet'>), ('ElasticNetCV', <class 'sklearn.linear_model._coordinate_descent.ElasticNetCV'>), ('ExtraTreeRegressor', <class 'sklearn.tree._classes.ExtraTreeRegressor'>), ('GradientBoostingRegressor', <class 'sklearn.ensemble._gb.GradientBoostingRegressor'>), ('HistGradientBoostingRegressor', <class 'sklearn.ensemble._hist_gradient_boosting.gradient_boosting.HistGradientBoostingRegressor'>), ('HuberRegressor', <class 'sklearn.linear_model._huber.HuberRegressor'>), ('KNeighborsRegressor', <class 'sklearn.nei

  3%|▎         | 1/35 [00:21<12:27, 21.97s/it]

{'Model': 'AdaBoostRegressor', 'R-Squared': 0.09793737007879566, 'Adjusted R-Squared': 0.09783532127383276, 'RMSE': np.float64(35.068352888777696), 'Time taken': 21.972631692886353}


...

 94%|█████████▍| 33/35 [1:04:19<02:17, 68.72s/it]

{'Model': 'TheilSenRegressor', 'R-Squared': 0.008817407031186053, 'Adjusted R-Squared': 0.008754031873386747, 'RMSE': np.float64(1485.3832805091156), 'Time taken': 192.73929858207703}


100%|██████████| 35/35 [1:04:20<00:00, 110.30s/it]

{'Model': 'TransformedTargetRegressor', 'R-Squared': 0.14319914394933853, 'Adjusted R-Squared': 0.14314436101646622, 'RMSE': np.float64(1381.025328757486), 'Time taken': 1.1042931079864502}
VotingRegressor model failed to execute
VotingRegressor.__init__() missing 1 required positional argument: 'estimators'


,Adjusted R-Squared,R-Squared,RMSE,Time Taken,Target
Model,,,,,
HistGradientBoostingRegressor,0.22,0.22,1313.52,3.29,PumpSecondsOnSite_mean
RandomForestRegressor,0.22,0.22,1316.24,64.31,PumpSecondsOnSite_mean
BaggingRegressor,0.22,0.22,1316.51,7.01,PumpSecondsOnSite_mean
MLPRegressor,0.22,0.22,1317.23,681.24,PumpSecondsOnSite_mean
ExtraTreeRegressor,0.22,0.22,1318.25,1.15,PumpSecondsOnSite_mean
DecisionTreeRegressor,0.22,0.22,1319.11,1.20,PumpSecondsOnSite_mean
GradientBoostingRegressor,0.21,0.21,1325.80,33.21,PumpSecondsOnSite_mean
AdaBoostRegressor,0.14,0.14,1380.19,6.35,PumpSecondsOnSite_mean
LassoLars,0.14,0.14,1381.02,0.44,PumpSecondsOnSite_mean
